# Module

In [1]:
from glob import glob
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
from tqdm import tqdm
import os

from ultralytics import YOLO

# 경로

In [3]:
imgs = sorted(glob('/workspace/ojt/OCR/dataset/infer_final/image/*'))

# 모형

In [5]:
model = YOLO('/workspace/ojt/OCR/YOLO/runs/detect/best_yolov8n_add/weights/best.pt')

# 추론

In [6]:
#results = model(imgs)
results = model(imgs)

/root/miniconda3/envs/yolo/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1716905971873/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,


0: 4096x4096 42 Texts, 20.0ms
1: 4096x4096 220 Texts, 20.0ms
2: 4096x4096 298 Texts, 20.0ms
3: 4096x4096 132 Texts, 20.0ms
4: 4096x4096 300 Texts, 20.0ms
Speed: 59.9ms preprocess, 20.0ms inference, 12.6ms postprocess per image at shape (1, 3, 4096, 4096)


# 시각화

In [ ]:
im_bgr = results[4].plot(labels=False)  # BGR-order numpy array
im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image
im_rgb

In [7]:
# Visualize the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot(labels=False)  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    r.save(filename=f"/workspace/ojt/OCR/dataset/infer_final/detect_result/result3{i+1}.jpg",labels=False)
  

# Create text recognition dataset

In [28]:
for i in range(1):
    boxes = results[i].boxes.xyxy.cpu().numpy()
    sample = cv2.imread(imgs[i])
    sample = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)
    name = imgs[i].split('/')[-1][:-4]

    for j in range(len(boxes)):
        min_x, min_y, max_x, max_y = int(boxes[j][1]), int(boxes[j][0]), int(boxes[j][3]), int(boxes[j][2])
        patch = sample[min_x : max_x + 1, min_y : max_y + 1,:]
        cv2.imwrite(f'/workspace/ojt/OCR/dataset/infer/text_data/{name}_text_{j+1}.png', patch)